# Tutorial for OpenAI API
目次
- OpenAI APIのインストール
- 環境変数の設定
- シンプルな文章生成
- オープンデータセットを用いた文章分類
- 課題1. 文章分類タスクの評価
- Extra課題

注意点
- この演習はGoogle Colaboratyで実行することを想定しています

## OpenAI APIのインストール
- [公式ドキュメント](https://platform.openai.com/docs/api-reference?lang=python)
- [GitHub](https://github.com/openai/openai-python)

In [ ]:
# ライブラリのインストール
!pip install openai==1.3.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


## 環境変数の設定
左のタブの`シークレット`から`OPENAI_API_KEY`を設定


<img width=500 src="https://github.com/HarmoLab/aitr/blob/main/2023/exercise_04/colab_secret.png?raw=true">

In [ ]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(
    api_key= userdata.get('OPENAI_API_KEY')
)

In [ ]:
# promptから料金を出力する関数
def calculate_credit(model:str, input_tokens:int, output_tokens:int) -> float:
    yen_rate = 150 # $1 = 150円で計算
    if model == "gpt-3.5-turbo-1106":
        input_credit = (0.001 * yen_rate) / 1000
        output_credit = (0.002 * yen_rate) / 1000
    elif model == "gpt-4-1106-preview":
        input_credit = (0.01 * yen_rate) / 1000
        output_credit = (0.03 * yen_rate) / 1000
    elif model == "gpt-4-turbo":
        input_credit = (0.01 * yen_rate) / 1000
        output_credit = (0.03 * yen_rate) / 1000
    elif model == "o1-preview":
        input_credit = (0.015 * yen_rate) / 1000
        output_credit = (0.06 * yen_rate) / 1000
    elif model == "o1-mini":
        input_credit = (0.003 * yen_rate) / 1000
        output_credit = (0.012 * yen_rate) / 1000
    elif model == "gpt-4o-mini-2024-07-18":
        input_credit = (0.0015 * yen_rate) / 1000
        output_credit = (0.003 * yen_rate) / 1000
    else:
        assert False, f"モデル名: {model}の料金計算はできません。modelは'gpt-3.5-turbo-1106',または'gpt-4-1106-preview'を使用してください。"
    total_credit = round(input_credit * input_tokens + output_credit * output_tokens, 2)
    print(f"使用したモデル: {model}, 料金: {total_credit}円")
    return total_credit


## シンプルな文章生成
- 公式ドキュメント
    - 文章生成(text-generation): https://platform.openai.com/docs/guides/text-generation


In [ ]:
user_prompt = "ChatGPTについて簡潔に3文で教えてください。" # ここを修正
model = "gpt-3.5-turbo-1106"

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model=model,
    max_tokens=500,
    n=1,
    temperature=1,

)

In [ ]:
print("--- レスポンス全体 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)

--- レスポンス全体 ---
ChatCompletion(id='chatcmpl-AQ5DFhxY3kkYIgYmheIPzg1ed6Jwr', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='ChatGPTは、自然言語処理を用いた会話エージェントです。人工知能を活用して、ユーザーとの会話を通じて質問に答えたり、情報を提供したりします。また、さまざまなトピックについて深い会話を行うことができます。', role='assistant', function_call=None, tool_calls=None, refusal=None), logprobs=None)], created=1730779929, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_e7d4a5f731', usage=CompletionUsage(completion_tokens=95, prompt_tokens=29, total_tokens=124, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))
使用したモデル: gpt-3.5-turbo-1106, 料金: 0.03円

--- 質問 ---
ChatGPTについて簡潔に3文で教えてください。
--- 回答 ---
ChatGPTは、自然言語処理を用いた会話エージェントです。人工知能を活用して、ユーザーとの会話を通じて質問に答えたり、情報を提供したりします。また、さまざまなトピックについて深い会話を行うことができます。


### chat.completions.createに使用する引数について
公式ドキュメント: https://platform.openai.com/docs/api-reference/chat/create

### 主要な引数
- model (必須)
    - 使用する学習モデル (本演習では"gpt-3.5-turbo-1106"を使用)
- messages (必須)
    - role
        - system: アシスタントの動作を設定
        - assistant: アシスタントの望ましい動作を設定 (ユーザも作成可)
        - user: ユーザの指示
    - content
        - roleに対して入力する文章
- max_tokens: int or null
    - 生成するトークンの最大数。出力の長さを制限することが可能
- n: int or null
    - 生成するレスポンスの数 (デフォルト: 1)
- temperature: number or null
    - 0～2の間で指定
    - 0に近づくにつれて決定論的になり、2に近づくにつれて生成文が多様でランダムになる（デフォルト: 1）
        - 詳しい説明: https://techblog.a-tm.co.jp/entry/2023/04/24/181232

## オープンデータセットを用いた文章分類
- 今回使用するデータセット
    - Amazon Reviews Multi:
https://huggingface.co/datasets/mteb/amazon_reviews_multi/viewer/ja

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import datasets
# Amazonレビューのデータセット(日本語)をダウンロード
# 今回はダウンロード時間を減らすために検証データ (validation)のみ取得
dataset = datasets.load_dataset("mteb/amazon_reviews_multi", "ja", split="validation")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

amazon_reviews_multi.py:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/40.7M [00:00<?, ?B/s]

ja/validation/0000.parquet:   0%|          | 0.00/999k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 5000
})


In [ ]:
# 実際のデータセットを確認
dataset[0]["text"]

'不良品\n\n味自体及び吸い心地は良いのだが、不良品が多過ぎる。私の場合５本のうち２本が蒸気も出ず、吸い込み も出来なかった。腹が立ってごみ箱行きでした。こんなものは２度と購入する気はない。 返品するのも交渉するのも、金額も金額だからと面倒くさがってしない方が多いのではないか？ 最初から不良品多しとでも表記しておいたら如何？'

In [ ]:
system_prompt = "以下のレビュー文を評価してください"
user_prompt = dataset[0]["text"] # レビュー文のみ

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
print("--- 全体の出力 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)

--- 全体の出力 ---
ChatCompletion(id='chatcmpl-AQ5DwJxXieGi4uGq5Tmzdnp2CtBVH', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='このレビューは製品の味や吸い心地について肯定的な意見を述べていますが、不良品が多いという点でマイナスの評価をしています。不良品が多いという問題は重要な点であり、消費者にとっては非常に不満足な経験となるでしょう。また、返品や交渉が金額や手間の問題で煩わしく感じられるという指摘もあります。改善点としては、製品が不良品である可能性を事前に明記することで、消費者の選択をサポートすることが挙げられます。このレビューは消費者にとって参考になる情報を提供していますが、改善点も示唆されているので、評価は詳細かつバランスのとれたものと言えるでしょう。', role='assistant', function_call=None, tool_calls=None, refusal=None), logprobs=None)], created=1730779972, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=280, prompt_tokens=192, total_tokens=472, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))
使用したモデル: gpt-3.5-turbo-1106, 料金: 0.11円

--- 質問 ---
不良品

味自体及び吸い心地は良いのだが、不良品が多過ぎる。私の場合５本のうち２本が蒸気も出ず、吸い込み も出来なかった。腹が立ってごみ箱行きでした。こんなものは２度と購入す

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# forでループするために関数化 (RateLimit対策のためtenacityのデコーダを採用)
# 参考: https://cookbook.openai.com/examples/how_to_handle_rate_limits
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def simple_completion(model_name, system_prompt:str, user_prompt:str):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            },
        ],
        model=model_name,
        timeout=15
    )
    # 料金の計算
    credit = calculate_credit(model_name, response.usage.prompt_tokens, response.usage.completion_tokens)
    return response, credit

## 課題1. 文章分類タスクの評価
system_promptを変更してレビュー文章を用いた評価値推定を行いましょう。

In [ ]:
model_name = "gpt-4o-mini-2024-07-18" # モデル名
k = 30 # k件のレビュー文に対する評価
shuffled_dataset = dataset.shuffle(seed=70) # データセットをシャッフル
review_list = shuffled_dataset[:k]["text"]
label_list = shuffled_dataset[:k]["label"]
print(review_list)
print(label_list)

# 課題1: 正答率と評価値を上げるためのsystem_promptを考えましょう。
system_prompt = """
以下のレビュー文を5段階で評価してください。0~4の数字のみを出力してください。
"""
system_prompt = """
レビュー文を0から4までの整数に分類してください。

### 例 ###
レビュー文: ヒト由来の乳酸菌
体にもよさそうで値段も安いので喜んで食べてましたが、ふと、「ヒト由来」って何よ？と思い調べてみました。 製品ホームページにも詳しくは書いてないですが、 実は・・・から採取、分離、培養してるそうです。 お客様相談室にも確認しました。 うーーん。
2

レビュー文: お得感。ありました。温かそうで可愛いです。3.1kgチワワでMサイズで大丈夫でした。梱包も到着日も満足しています。
3

レビュー文: 商品がちゃんと届きません！定期お届け便でこの商品を購入しましたが、とにかく配送業者がいい加減で在宅してるのに不在票だけポストに入れて帰る。再配達依頼の電話に出ない。電話に出ても「はい。行きます。」と言ったっきり来ない等々で今月２１日到着予定が２７日になっても届きません。amazonに確認したところ配送業者の変更は出来ないとの事でしたのでクレーム入れてキャンセルしました。 どんなに良い商品でも届かないと意味がありません！
0

レビュー文: 梱包丁寧。メダカ元気。何匹かは到着した時には死んでいると思っていたけどそんな事はなく凄く元気でした。 今は卵から赤ちゃんが誕生しています。
3
"""

predict_label_list = [] # -1(出力形式のエラー), 0, 1, 2, 3, 4
total_credit = 0
for i, (review, label) in enumerate(zip(review_list, label_list)):
    print(f"\n^^^^^^^^^^ {i+1}件目のレビュー ^^^^^^^^^^")
    print("--- レビュー文 ---")
    print(review)
    print("--- 実際の評価 ---")
    print(label)
    response, credit = simple_completion(model_name, system_prompt, review)
    total_credit += credit
    print("--- ChatGPTの回答 ---")
    print(response)
    print(response.choices[0].message.content)

    try:
        response_int = int(response.choices[0].message.content)
        predict_label_list.append(response_int)
    except ValueError:
        print("--- Error ---")
        print("回答がint型に変換できませんでした。")
        predict_label_list.append(-1)
        continue
print(f"合計金額: {total_credit}円")

['無難な一品\n\nC26セレナに装着。風切り音無し。 キャンプ道具積載量アップ。 ミニバンだと乗せおろしが大変。 スタンダードで無難。', '柔らかめの生地感ですごく良かったです\n\n先ほど到着したばかりで、妻に新年プレゼントの為に買いました。柔らかめの生地感ですごく良かったです。お値段も高くないし、良かったです。今後も利用するので、よろしくお願いします。', '大好きな香り！\n\n部屋には定期的に噴射する他社の芳香剤を置いていますが、これは人感センサーなのでトイレに置いています！人感センサーなので、トイレを使用した時だけいい香りにしてくれて無駄がありませんし、匂いも私の大好きな匂いでトイレがとてと心地よく感じられます！ 香りの持続力はありませんが、トイレに入ったら瞬時に噴射してくれるのでいつもいい香りをかいでます。', '品質が悪くなった！？\n\nリピーターです。品質が落ちたと思います。浴室に取付けて3ヶ月、既に鏡の淵の裏側から腐食が見られます。また、家族の身長によって上下に向きを変えるのですが、たびたび吸盤から鏡を止めているプラスチックボール部が外れて落下します。以前の物も落下はありましたが、吸盤のボール受け部分が浅くなったのか、ボールが大きくなったのかは古い方を捨てたので比較出来ませんが、新しいのにもかかわらず頻繁に落ちます。強く押し込んでも浮いて見えます。鏡が割れそうなほど強く押し込んでいるのですが。 まあ、落下しても淵のプラスチックの曲面が上手くショックを吸収するように作られている様で、以前のものは鏡自体は最後まで割れませんでした。経年で淵のプラスチックが硬化し、落下のたびにプラスチックの淵が割れていきました。ちなみに、浴室乾燥機はなく直射日光も当たらない環境での利用です。', 'スピード、名作です\n\nかなりの値引き時に購入。もともとDVD持っていたのですが、セットだったので再購入。満足の採用です。', 'コスパ良き\n\n物はいいけど３つ中１つの支えの１つが割れていた。ショートのウィッグを掛けるぶんには問題ないのでそのままつかっているけれど、少し残念なので星3。割れていない２つに関してはとても良い。総合的に見て、たまたま私が外れを引いたとしてもコスパは良いと思う。', '値段相応\n\n予想範囲内のコピー品です。予想の範囲内なので可ですが。

In [ ]:
print("--- 実際の評価 ---")
print(label_list)
print("--- 予測した評価--- ")
print(predict_label_list)

# 評価
## 実際の評価と予測した評価の差の合計を評価値 (loss)
## 予測した評価が-1（出力形式のエラー）の場合は+6

validation_score = 0 # 評価値
correct_answer = 0 # 正解数
for actual, predict in zip(label_list, predict_label_list):
    if not actual == predict:
        if predict == -1:
            validation_score += 6
            continue
        validation_score += abs(actual - predict)
    else:
        correct_answer += 1
print("\n--- 正答率 ---")
print(f"{correct_answer/len(label_list)*100} %")
print("--- 評価値 (低いほうが良い結果) ---")
print(validation_score)

--- 実際の評価 ---
[2, 4, 3, 1, 4, 2, 1, 2, 3, 2, 2, 3, 0, 3, 3, 2, 3, 2, 4, 0, 1, 1, 4, 1, 0, 1, 4, 0, 3, 1]
--- 予測した評価--- 
[2, 4, 3, 1, 4, 3, 1, 2, 3, 3, 2, 3, 0, 3, 4, 3, 3, 3, 3, 1, -1, 1, -1, 1, 1, 1, 4, 0, 3, 1]

--- 正答率 ---
66.66666666666666 %
--- 評価値 (低いほうが良い結果) ---
20


## Extra課題


In [ ]:
user_prompt = "2泊3日の札幌観光旅行のプランを考えてください。"

# Extra課題
system_prompt = """
ユーザの質問に回答してください。
"""

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
print("--- 全体の出力 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)